In [13]:
import pandas as pd
import numpy as np

import unicodedata
import re
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
def bible_wrangle():
    df = pd.read_csv('t_asv.csv')
    df_genre = pd.read_csv('key_genre_english.csv')
    df_book_names = pd.read_csv('key_english.csv')
    df = pd.merge(df,df_book_names,left_on = 'b',right_on = 'field',how = 'left')
    df.drop(columns='field',inplace=True)
    df = pd.merge(df,df_genre,left_on = 'field.3',right_on='field',how='left')
    df.drop(columns='field',inplace=True)
    df.columns = ['id','book_no','ch','ver','text','book','test','genre_no','genre']
    return df

In [3]:
df = bible_wrangle()

In [8]:
def basic_clean(string):
    """
    Lowercase the string
    Normalize unicode characters
    Replace anything that is not a letter, number, whitespace or a single quote.
    """
    string = string.lower()
    string = string.replace('c++','cplusplus')
    string = unicodedata.normalize('NFKD', string).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # remove anything not a space character, an apostrophy, letter, or number
    string = re.sub(r"[^a-z0-9'\s]", '', string)

    # convert newlines and tabs to a single space
    string = re.sub(r'[\r|\n|\r\n]+', ' ', string)
    
    string = string.strip()
    return string

In [9]:
df.head(2)

,id,book_no,ch,ver,text,book,test,genre_no,genre
0,1001001,1,1,1,In the beginning God created the heavens and t...,Genesis,OT,1,Law
1,1001002,1,1,2,And the earth was waste and void; and darkness...,Genesis,OT,1,Law


In [11]:
df.text = df.text.apply(basic_clean)

In [12]:
df.head(2)

,id,book_no,ch,ver,text,book,test,genre_no,genre
0,1001001,1,1,1,in the beginning god created the heavens and t...,Genesis,OT,1,Law
1,1001002,1,1,2,and the earth was waste and void and darkness ...,Genesis,OT,1,Law


In [15]:
sia = SentimentIntensityAnalyzer()

In [20]:
blob = "For God so loved the world that he gave his only son to die for us"

In [21]:
sia.polarity_scores(blob)

{'neg': 0.163, 'neu': 0.542, 'pos': 0.296, 'compound': 0.4924}

In [23]:
df['vader_sentiment'] = df.text.apply(lambda txt: sia.polarity_scores(txt)['compound'])

In [32]:
df.groupby('genre').mean().sort_values(by='vader_sentiment',ascending=False)

,id,book_no,ch,ver,genre_no,vader_sentiment
genre,,,,,,
Epistles,5.135178e+07,51.346098,5.669075,13.190751,7.0,0.203782
Apocalyptic,6.601172e+07,66.000000,11.710396,10.358911,8.0,0.202125
Acts,4.401426e+07,44.000000,14.243297,19.732870,6.0,0.086857
Wisdom,1.918203e+07,19.133751,48.259143,15.790178,3.0,0.072520
Gospels,4.149854e+07,41.486372,12.148981,23.724001,5.0,0.049409
History,1.113957e+07,11.126674,12.877885,17.224850,2.0,0.046005
Law,2.837072e+06,2.816644,20.409433,18.417806,1.0,0.028739
Prophets,2.620389e+07,26.181967,21.906011,13.780692,4.0,-0.010489
